In [ ]:
#import necessary packages
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import seaborn as sns
import datetime
import warnings 
warnings.simplefilter('ignore') 

#load map packages
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from calendar import month_abbr

In [ ]:
ddir = '../saildrone_data/'  #data 2
fn = 'saildrone_west_coast_survey_2019_sd1047_JPLv5.0_8dy_20210613norep_20210613_20211116.nc'

In [ ]:
#open .nc file with Saildrone Data
sail = xr.open_dataset(ddir+fn) 
sail.close()

#copy data for calculations versus graphing
sail_raw = sail.copy()

In [ ]:
#resample the data by Day
resampled_temp = sail_raw.resample(time="D", skipna = True).mean()

#set figure size
plt.figure(figsize = (14,4))

#create lineplot 
sns.lineplot(x = sail_raw['time'].dt.date, y = sail_raw['TEMP_CTD_MEAN'], color = 'grey', ci = 'sd', alpha=0.2)
plt.scatter(resampled_temp['time'].dt.date, resampled_temp['TEMP_CTD_MEAN'], 30, [*range(len(resampled_temp['TEMP_CTD_MEAN']))], 'o', cmap='jet')
plt.tight_layout()

#set labels
plt.title('Water Temperature for Saildrone Cruise', fontdict = {'fontsize' : 16})
plt.xlabel("Date")
plt.ylabel("Temperature [C]")

#reformat x-axis ticks
plt.grid(zorder=0, alpha=0.5)
plt.show()

In [ ]:
#define latitude and longitude boundaries
latr = [min(sail_raw['lat']), max(sail_raw['lat'])] 
lonr = [max(sail_raw['lon']), min(sail_raw['lon'])] 

# Select a region of our data, giving it a margin
margin = 0.5 
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]+margin,lonr[1]-margin]]) 

#add state outlines
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

# Create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree()) 
ax.coastlines(resolution='10m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(states_provinces, linewidth = 0.5)
ax.add_feature(cfeature.BORDERS)
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.set_xticks(np.round([*np.arange(region[1,1],region[1,0]+1,2)][::-1],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0]),region[0,1]+1,1.5)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.gridlines(linestyle = '--', linewidth = 0.5)

# Plot track data, color by temperature
sc = plt.scatter(data = sail_raw, x = 'lon', y = 'lat', c = 'TEMP_CTD_MEAN')
clb = fig.colorbar(sc, label = "Temperature [C]")
plt.set_cmap('Reds')
plt.title('Raw Temperature Values', fontdict = {'fontsize' : 16})
plt.show()